<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/05_Noy_2024_home_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create KNN ensemble model

## Import

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt

## Load `train.csv`

In [2]:
df = pd.read_csv('train.csv')
df.head(1)

,id,Row#,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,yield
0,0,104.0,12.5,0.25,0.25,0.25,0.75,69.7,42.1,58.2,50.2,24.3,41.2,16.0,0.26,0.477941,0.423927,34.043022,6079.08526


## Split data

In [3]:
X = df.drop(['yield'], axis=1)
y = df['yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Functions

In [7]:
def print_metrics(model, X_train, X_test, y_train, y_test, cv_mae):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    r2 = r2_score(y_test, y_test_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

    text = f"""
    Model Performance Metrics:
    --------------------------
    CV Mean MAE: {-cv_mae.mean():.2f}, {-cv_mae.std():.2f}

           Train  | Test
         |-----------------|
     MAE | {train_mae:.2f} | {test_mae:.2f} |
         |-----------------|
    RMSE | {train_rmse:.2f} | {test_rmse:.2f} |
         |-----------------|

        R^2 Score (Test): {r2:.2f}
    """
    print(text)

## Initialize stacking model

In [9]:
estemators = [
    ('knn',Pipeline([
        ('scaler', StandardScaler()),
        ('regressor',  KNeighborsRegressor(n_neighbors=10))])),
    ('dt', DecisionTreeRegressor(max_depth=5)),
    ('rf', RandomForestRegressor(max_depth=5)),
    ('lr', Pipeline([
        ('scaler', StandardScaler()),
        ('regressor',  LinearRegression())]))
]

final_estimator = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor',  KNeighborsRegressor(n_neighbors=10))])

stacking_model = StackingRegressor(estimators=estemators,
                                   final_estimator=final_estimator,
                                   passthrough=True)

cv = KFold(n_splits=3, shuffle=True, random_state=42)
neg_mae = cross_val_score(stacking_model, X, y, cv=cv, scoring='neg_mean_absolute_error')
stacking_model.fit(X_train, y_train)

StackingRegressor(estimators=[('knn',
                               Pipeline(steps=[('scaler', StandardScaler()),
                                               ('regressor',
                                                KNeighborsRegressor(n_neighbors=10))])),
                              ('dt', DecisionTreeRegressor(max_depth=5)),
                              ('rf', RandomForestRegressor(max_depth=5)),
                              ('lr',
                               Pipeline(steps=[('scaler', StandardScaler()),
                                               ('regressor',
                                                LinearRegression())]))],
                  final_estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                                  ('regressor',
                                                   KNeighborsRegressor(n_neighbors=10))]),
                  passthrough=True)

In [10]:
print_metrics(stacking_model, X_train, X_test, y_train, y_test, neg_mae)


    Model Performance Metrics:
    --------------------------
    CV Mean MAE: 315.07, -1.36

           Train  | Test
         |-----------------|
     MAE | 275.92 | 305.80 |
         |-----------------|
    RMSE | 388.08 | 425.31 |
         |-----------------|

        R^2 Score (Test): 0.90
    
